In [3]:
import pandas as pd
from pypdf import PdfReader 

In [4]:
reader = PdfReader('Mani_kumar_Bachelor_Thesis.pdf') 
print(len(reader.pages)) 
page = reader.pages[0] 
print(page.extract_text()) #Print the first page of the pdf file

40
The Yeast Connectome:
Constructing a Connectome Database
from Scientific Literature using GPT
Models
A Thesis submitted for the completion of
requirements for the degree of
Bachelor of Science
(Research)
by
Mani Kumar R
Undergraduate Programme
Indian Institute of Science
Under the supervision of
Prof. Marek Mutwil
School of Biological Sciences, Nanyang Technological University,
Singapore
Prof. Mohit Kumar Jolly
Department of Bioengineering, Indian Institute of Science


In [5]:
text=""
for i in range(len(reader.pages)): 
    page = reader.pages[i]
    text+=page.extract_text()

In [19]:
print(len(text))
sentences = text.split(".")
print(len(sentences))
print(sentences[555])

48737
1310
 INTRODUCTION 3
Large Language Models have demonstrated superior capabilities in natural language
processing tasks and beyond


In [7]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [9]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings/vectors


c:\Users\ksrma\anaconda3\envs\ml\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ksrma\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\ksrma\anaconda3\envs\ml\lib\site-packages\transformers\tokenization_utils_base.py:16

In [22]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance
qdrant.recreate_collection(
    collection_name="Bachelor_Thesis",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\ksrma\AppData\Local\Temp\ipykernel_5312\1529181893.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [23]:
# vectorize!
qdrant.upload_points(
    collection_name="Bachelor_Thesis",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(sentence),  # Encode sentence to vector
            payload={"content":sentence},
        ) for idx, sentence in enumerate(sentences) 
    ]
)

In [25]:
#function to Search in Pdf 
def search_in_pdf(query:str):
    hits = qdrant.search(
        collection_name="Bachelor_Thesis",
        query_vector=encoder.encode(query).tolist(),
        limit=3
    )
    return [hit.payload for hit in hits]


query="Number of researh articles processed?"
print(search_in_pdf(query))

[{'content': ' we processed\nmore than 84,427 research abstracts and full text articles from leading journals(Figure\n3'}, {'content': ' Then, We evaluated the fine-tuned model based on it’s performance on\n20 random articles'}, {'content': ' We processed 84,427 publications (Figure 3'}]


In [28]:
# Now time to connect to the local large language model

from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"    Your llamaserver url
    api_key = "sk-no-key-required"
)

user_prompt="In the Bachelor Thesis, what is the number of research articles processed?"

completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, Your top priority is to help guide users with their requests."},
        {"role": "user", "content":user_prompt },
        {"role": "assistant", "content": str(search_in_pdf(user_prompt))}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Yes, the number of research articles processed by the research team is more than 84,427.</s>', refusal=None, role='assistant', function_call=None, tool_calls=None)
